**Simulating Manipulator Control**

We want to simulate the way that we will interact with the manipulator to practice:

1. Moving it in x,y,z,r1,r2 space
2. Saving the current position in a dictionary by name
3. Moving to that position by name



In [1]:
from ophyd import Device, PVPositioner
from ophyd.sim import FakeEpicsSignal, FakeEpicsSignalRO
from ophyd import Component as Cpt

In [45]:
from ophyd.sim import SynAxis

class ManipulatorSim(Device):
    
    x = Cpt(SynAxis,name='x')
    y = Cpt(SynAxis,name='y')
    z = Cpt(SynAxis,name='z')
    r1 = Cpt(SynAxis,name='r1')
    r2 = Cpt(SynAxis,name='r2')
    
    pos_dict={}
    pos_dict['zero'] = [0,0,0,0,0]
    
    
    #save the current position with a particular name
    def save(self, name):
        
        self.pos_dict[name]= [self.x.readback.get(),
                              self.y.readback.get(),
                              self.z.readback.get(),
                              self.r1.readback.get(),
                              self.r2.readback.get()]
    
    def move_to(self, name):
        
        if name in self.pos_dict:
            
            self.x.setpoint.set(self.pos_dict[name][0])
            self.y.setpoint.set(self.pos_dict[name][1])
            self.z.setpoint.set(self.pos_dict[name][2])
            self.r1.setpoint.set(self.pos_dict[name][3])
            self.r2.setpoint.set(self.pos_dict[name][4])
            print(f'Manipulator at {name}')
        else:
            
            print(f'No position names {name} in position list')

es_man_sim = ManipulatorSim(name = 'es_man_sim')
ana_man_sim = ManipulatorSim(name = 'ana_man_sim')



In [46]:
# Now lets move the manipulator somewhere

es_man_sim.x.setpoint.set(2)
es_man_sim.y.setpoint.set(5)
es_man_sim.z.setpoint.set(2)
es_man_sim.r1.setpoint.set(1)
es_man_sim.r2.setpoint.set(2)

Status(obj=_SetpointSignal(name='es_man_sim_r2_setpoint', parent='es_man_sim_r2', value=2.0, timestamp=1634148490.4573238), done=False, success=False)

In [47]:
# We can call the save method to name this position
es_man_sim.save('sampleA')

In [48]:
es_man_sim.move_to('home')

No position names home in position list


In [49]:
es_man_sim.move_to('zero')

Manipulator at zero


In [50]:
#Now we can move to that sample position again
es_man_sim.move_to('sampleA')

Manipulator at sampleA


In [51]:
es_man_sim.read()

OrderedDict([('es_man_sim_x', {'value': 2.0, 'timestamp': 1634148496.5284214}),
             ('es_man_sim_x_setpoint',
              {'value': 2.0, 'timestamp': 1634148496.5272636}),
             ('es_man_sim_y', {'value': 5.0, 'timestamp': 1634148496.5290315}),
             ('es_man_sim_y_setpoint',
              {'value': 5.0, 'timestamp': 1634148496.528246}),
             ('es_man_sim_z', {'value': 2.0, 'timestamp': 1634148496.5297337}),
             ('es_man_sim_z_setpoint',
              {'value': 2.0, 'timestamp': 1634148496.5294464}),
             ('es_man_sim_r1',
              {'value': 1.0, 'timestamp': 1634148496.5306563}),
             ('es_man_sim_r1_setpoint',
              {'value': 1.0, 'timestamp': 1634148496.5302124}),
             ('es_man_sim_r2', {'value': 2.0, 'timestamp': 1634148496.53143}),
             ('es_man_sim_r2_setpoint',
              {'value': 2.0, 'timestamp': 1634148496.5308244})])

In [56]:
#If we want to move the motors in parallel we need to do somthing different. 

from bluesky import RunEngine

RE = RunEngine({})

from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

#Get the databroker https://gist.github.com/untzag/53281819709b1058c7708a250cbd3676
from databroker.v2 import temp
db = temp()

# Insert all metadata/data captured into db.
RE.subscribe(db.v1.insert)



1

In [57]:
#We can move things in parallel with the Bluesky stub plan mv

from bluesky.plan_stubs import mv

RE(mv(es_man_sim.x, 2.0, es_man_sim.y, 1.0))

()

In [58]:
es_man_sim.read()

OrderedDict([('es_man_sim_x', {'value': 2.0, 'timestamp': 1634148763.295282}),
             ('es_man_sim_x_setpoint',
              {'value': 2.0, 'timestamp': 1634148763.2949808}),
             ('es_man_sim_y', {'value': 1.0, 'timestamp': 1634148763.29574}),
             ('es_man_sim_y_setpoint',
              {'value': 1.0, 'timestamp': 1634148763.2954946}),
             ('es_man_sim_z', {'value': 2.0, 'timestamp': 1634148496.5297337}),
             ('es_man_sim_z_setpoint',
              {'value': 2.0, 'timestamp': 1634148496.5294464}),
             ('es_man_sim_r1',
              {'value': 1.0, 'timestamp': 1634148496.5306563}),
             ('es_man_sim_r1_setpoint',
              {'value': 1.0, 'timestamp': 1634148496.5302124}),
             ('es_man_sim_r2', {'value': 2.0, 'timestamp': 1634148496.53143}),
             ('es_man_sim_r2_setpoint',
              {'value': 2.0, 'timestamp': 1634148496.5308244})])

In [63]:
#Let's make a plan that moves things in parallel

def move_man_to_sample(man, name, *, md=None):
    
    if name in man.pos_dict:
        yield from mv(man.x,man.pos_dict[name][0],
                      man.y,man.pos_dict[name][1],
                      man.z,man.pos_dict[name][2],
                      man.r1,man.pos_dict[name][3],
                      man.r2,man.pos_dict[name][4])
    
    else:
        print(f'No position names {name} in position list for this manipulator')
        

In [64]:
RE(move_man_to_sample(es_man_sim,'zero'))

()

In [ ]:
# What are some benifits and problems with our implementation?